In [5]:
from FetoFlow import *
import pandas as pd

In [6]:
# read in node and element files
node_filename= '../sample_geometry/FullTree.ipnode'
element_filename = '../sample_geometry/FullTree.ipelem'

In [7]:
# process nodes and elements using functions from file_parsing_utils()
nodes = read_nodes(node_filename)
elements = read_elements(element_filename)

In [8]:
# define boundary conditions
inlet_pressure, outlet_pressure = 6650, 2660
# generate boundaray conditions dictionary 
print("Creating Boundary Conditions")
bcs = generate_boundary_conditions(inlet_pressure = inlet_pressure, outlet_pressure = outlet_pressure, inlet_flow=None)

Creating Boundary Conditions


In [9]:
# define other required geometric features (radii and decay factors)
umbilical_artery_radius, decay_factor = 1.8 / 1000, 1.38 
umbilical_vein_radius, decay_factor_vein = 4.0 / 1000, 1.46
arteries_only = False # this should rarely be true

viscosity_type = 'constant' # can also be 'pries_network' or 'pries_vessel' if wanting to incorporate radius-dependence

In [10]:
# Generate the di-graph & calculate the resistances based on the viscosity
print("Creating Geometry")
G = create_geometry(nodes, elements, umbilical_artery_radius, decay_factor, arteries_only, umbilical_vein_radius, decay_factor_vein)

Creating Geometry


In [11]:
print("Calculating Resistance")
G = calculate_resistance(G, viscosity_model=viscosity_type)

Calculating Resistance


In [12]:
# solve
print("Calculating Matrices")
A,b,bc_export = create_small_matrices(G,bcs,branching_angles=False)
print("Solving for Pressures and Flows")
p,q = solve_small_system(A,b,G,bc_export)

Calculating Matrices
Solving for Pressures and Flows


In [13]:
# store pressure and flows
pressures = pd.DataFrame([{"Node" : node, "Pressure" : pressure} for node,pressure in p.items()])
pressures.to_csv("../output_data/example_simulation_pressures.csv")
flows = pd.DataFrame([{"Element" : element, "Flow" : flow} for element,flow in q.items()])
flows.to_csv("../output_data/example_simulation_flows.csv")

In [14]:
pressures.head(10)

,Node,Pressure
0,0,6650.000000
1,1,6635.869604
2,2,6594.390526
3,3,6591.936575
4,4,6428.067478
5,5,6415.773662
6,6,6278.472676
7,7,6154.431935
8,8,6368.023392
9,9,6325.675841


In [15]:
flows.head(10)

,Element,Flow
0,0,0.000006
1,1,0.000003
2,2,0.000003
3,3,0.000003
4,4,0.000003
5,5,0.000001
6,6,0.000001
7,7,0.000002
8,8,0.000001
9,9,0.000001


In [16]:
node_filename = "../sample_geometry/FullTree.ipnode"
element_filename = "../sample_geometry/FullTree.ipelem"
boundary_conditions = {
    "inlet_pressure" : 6650,
    "outlet_pressure" : 2660
}
inlet_radius = 1.8/1000
strahler_ratio_arteries = 1.38

In [17]:
# call pressure and flows function
G = pressures_and_flows(
    node_filename,
    element_filename,
    boundary_conditions,
    inlet_radius,
    strahler_ratio_arteries,
    input_directory=".",
    output_directory="../output_data",
    flow_output_filename="flow_values.csv",
    pressure_output_filename="pressure_values.csv",
    arteries_only=False,
    viscosity_model="constant",
    vessel_type="rigid",
    outlet_vein_radius=4.0,
    strahler_ratio_veins=1.46,
    anastomosis=None,
    mu=0.33600e-2,  # This is the non-capillary viscosity value used
    capillary_model="analytical2015",
    capillary_parameters=None, 
    radius_filename=None,
    other_field_filenames=None,  
    verbose=False,
    time_statistics=False,
    return_graph=True,
)
print(G)

C:\Users\Jude\AppData\Local\Temp\ipykernel_18424\2516512620.py:2: UserWarning: No capillary parameters passed in. Using default values.
  G = pressures_and_flows(


DiGraph with 127216 nodes and 159017 edges
